In [4]:
# ── 설치 (최초 1회만)
!pip -q install -U langchain langchain-openai langsmith


In [ ]:
# 1) (선택) OpenAI 사용 시
import os, uuid
from dotenv import load_dotenv

load_dotenv()
# OpenAI API 클라이언트 생성
OPENAPI_KEY = os.getenv("OPENAI_API_KEY")
LangSmith_KEY = os.getenv("LANGCHAIN_API_KEY")

# 2) LangSmith 연동 필수 환경변수
os.environ["LANGCHAIN_TRACING_V2"] = "true"      # 트레이싱 활성화
os.environ["LANGSMITH_ENDPOINT"]   = "https://api.smith.langchain.com"  # 기본값
os.environ["LANGSMITH_PROJECT"]    = "llm_colab_ex_RunnablePassthrough"                 # 수업용 프로젝트명


In [6]:


# ── 임포트
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

# ── LLM 준비
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
parser = StrOutputParser()

# ── 프롬프트: 질문(question)과 컨텍스트(context)를 받아 답변
prompt = ChatPromptTemplate.from_template(
    "질문: {question}\n"
    "컨텍스트: {context}\n"
    "👉 위 내용을 바탕으로 간단히 답해줘."
)

# ── RunnablePassthrough를 사용한 체인
# question은 그대로 통과, context는 전처리 함수 적용
chain = (
    {
        "question": RunnablePassthrough(),  # 입력의 question을 그대로 전달
        "context": RunnableLambda(lambda x: f"[Extra Context Attached] {x['context']}")
    }
    | prompt
    | llm
    | parser
)

# ── 실행 (LangSmith 트래킹 태그/이름 추가)
config = {"tags":["demo","passthrough"], "run_name":"passthrough_demo"}

inputs = {"question": "LangSmith는 무엇인가요?", "context": "LangSmith는 LLM 앱을 관찰/평가/디버깅할 수 있는 플랫폼이다."}

print(chain.invoke(inputs, config=config))


LangSmith는 LLM 앱을 관찰, 평가, 디버깅할 수 있는 플랫폼입니다.
